# Chargement des librairies

In [1]:
import os
import sys

import logging
import random
from datetime import datetime
from pathlib import Path

!pip install implicit
import implicit

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

!pip install plotly
import plotly.express as px
import plotly.io as pio

from implicit.als import AlternatingLeastSquares

!pip install lightfm
from lightfm import LightFM

from pandas.api.types import is_numeric_dtype
from scipy import sparse
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from tqdm import tqdm

pio.renderers.default = "iframe"

# Plotly as Pandas plotting backend
pd.options.plotting.backend = "plotly"

# Fonctions utilitaires

In [2]:
def reduce_dataframe_memory_usage(
    df: pd.DataFrame,
    high_precision: bool = False,
) -> pd.DataFrame:
    """
    Iterate through all the columns of a dataframe and modify the data type to
    reduce memory usage.
    Args:
        df (pd.DataFrame): dataframe to reduce memory usage.
        high_precision (bool): If True, use 64-bit floats instead of 32-bit
    Returns:
        pd.DataFrame: dataframe with reduced memory usage.
    """
    start_mem = round(df.memory_usage().sum() / 1024**2, 2)
    logging.info("Memory usage of dataframe is %d MB", start_mem)

    # Iterate through columns
    for col in df.columns:
        if df[col].dtype == "object":
            # "object" dtype
            if df[col].nunique() < max(100, df.shape[0] / 100):
                # If number of unique values is less than max(100, 1%)
                df[col] = df[col].astype("category")
            else:
                # If number of unique values is greater than max(100, 1%)
                df[col] = df[col].astype("string")

        elif str(df[col].dtype)[:3] == "int":
            # "int" dtype
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df[col] = df[col].astype("UInt8")
            elif c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype("Int8")
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df[col] = df[col].astype("UInt16")
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype("Int16")
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df[col] = df[col].astype("UInt32")
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype("Int32")
            elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                df[col] = df[col].astype("UInt64")
            else:
                df[col] = df[col].astype("Int64")

        elif str(df[col].dtype)[:5] == "float":
            # "float" dtype
            c_min = df[col].min()
            c_max = df[col].max()
            if (
                not high_precision
                and c_min > np.finfo(np.float32).min
                and c_max < np.finfo(np.float32).max
            ):
                df[col] = df[col].astype("float32")
            else:
                df[col] = df[col].astype("float64")

    end_mem = round(df.memory_usage().sum() / 1024**2, 2)
    logging.info("Memory usage after optimization is %d MB", end_mem)
    if start_mem > 0:
        logging.info(
            "Decreased by %d %%", round(100 * (start_mem - end_mem) / start_mem)
        )

    return df

# Chargement des fichiers

In [3]:
DATA_PATH = "/storage/P9"

CACHE = dict()

NUM_EMBEDDINGS = 250

articles = pd.concat(
    [
        pd.read_csv(
            Path(DATA_PATH, "articles_metadata.csv"),
            parse_dates=["created_at_ts"],
            date_parser=lambda x: datetime.fromtimestamp(int(x) / 1000),
            dtype={
                "article_id": "category",
                "category_id": "category",
                "publisher_id": "category",
                "words_count": "int",
            },
        ),
        pd.DataFrame(
            pd.read_pickle(Path(DATA_PATH, "articles_embeddings.pickle")),
            columns=["embedding_" + str(i) for i in range(NUM_EMBEDDINGS)],
        ),
    ],
    axis=1,
).set_index("article_id")

articles = reduce_dataframe_memory_usage(
    articles.astype({"created_at_ts": "datetime64[ns]"})
)

articles_sample = articles.sample(frac=0.01, random_state=42)

articles.describe(include="all", datetime_is_numeric=True)

,category_id,created_at_ts,publisher_id,words_count,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,...,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249
count,364047,364047,364047,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,...,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000
unique,461,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,281,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,12817,NaN,364047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2016-09-16 23:57:17.328421888,NaN,190.897727,-0.238645,-0.963335,0.118548,-0.279295,-0.068579,0.045945,...,-0.133287,-0.081912,-0.060347,0.023003,0.076947,0.084603,0.062819,0.099768,0.155917,-0.041094
min,NaN,2006-09-27 11:14:35,NaN,0.000000,-0.991183,-0.996455,-0.968431,-0.994966,-0.994489,-0.985974,...,-0.990412,-0.989408,-0.990432,-0.993626,-0.989042,-0.996902,-0.992921,-0.984733,-0.976071,-0.988213
25%,NaN,2015-10-15 16:00:43.500000,NaN,159.000000,-0.620072,-0.974056,-0.289953,-0.718816,-0.503425,-0.354579,...,-0.547684,-0.445079,-0.479989,-0.404508,-0.248653,-0.267072,-0.306548,-0.313598,-0.201402,-0.420694
50%,NaN,2017-03-13 16:27:29,NaN,186.000000,-0.302581,-0.967605,0.124339,-0.391535,-0.093734,0.062636,...,-0.175781,-0.094113,-0.078034,0.000726,0.105649,0.133525,0.083315,0.128757,0.188355,-0.015232
75%,NaN,2017-11-05 14:09:11,NaN,218.000000,0.098015,-0.959061,0.545112,0.108320,0.345024,0.446979,...,0.250641,0.270006,0.341105,0.459386,0.417347,0.461466,0.441831,0.531453,0.538111,0.334226
max,NaN,2018-03-13 12:12:30,NaN,6690.000000,0.983694,-0.514728,0.998341,0.978092,0.996798,0.996343,...,0.996401,0.981789,0.991332,0.995299,0.978823,0.989324,0.991445,0.997583,0.990507,0.968462


In [4]:
clicks = pd.concat(
    [
        pd.read_csv(
            click_file_path,
            parse_dates=["session_start", "click_timestamp"],
            date_parser=lambda x: datetime.fromtimestamp(int(int(x) / 1000)),
            dtype={
                "user_id": "category",
                "session_id": "category",
                "session_size": "int",
                "click_article_id": "category",
                "click_environment": "category",
                "click_deviceGroup": "category",
                "click_os": "category",
                "click_country": "category",
                "click_region": "category",
                "click_referrer_type": "category",
            },
        ).replace(
            {
                "click_environment": {
                    "1": "1 - Facebook Instant Article",
                    "2": "2 - Mobile App",
                    "3": "3 - AMP (Accelerated Mobile Pages)",
                    "4": "4 - Web",
                },
                "click_deviceGroup": {
                    "1": "1 - Tablet",
                    "2": "2 - TV",
                    "3": "3 - Empty",
                    "4": "4 - Mobile",
                    "5": "5 - Desktop",
                },
                "click_os": {
                    "1": "1 - Other",
                    "2": "2 - iOS",
                    "3": "3 - Android",
                    "4": "4 - Windows Phone",
                    "5": "5 - Windows Mobile",
                    "6": "6 - Windows",
                    "7": "7 - Mac OS X",
                    "8": "8 - Mac OS",
                    "9": "9 - Samsung",
                    "10": "10 - FireHbbTV",
                    "11": "11 - ATV OS X",
                    "12": "12 - tvOS",
                    "13": "13 - Chrome OS",
                    "14": "14 - Debian",
                    "15": "15 - Symbian OS",
                    "16": "16 - BlackBerry OS",
                    "17": "17 - Firefox OS",
                    "18": "18 - Android",
                    "19": "19 - Brew MP",
                    "20": "20 - Chromecast",
                    "21": "21 - webOS",
                    "22": "22 - Gentoo",
                    "23": "23 - Solaris",
                },
            }
        )
        for click_file_path in tqdm(
            sorted(Path(DATA_PATH, "clicks").glob("clicks_hour_*.csv"))
        )
    ],
    sort=False,
    ignore_index=True,
    verify_integrity=True,
)

clicks = reduce_dataframe_memory_usage(
    clicks.astype(
        {
            "session_start": "datetime64[ns]",
            "session_size": "int",
            "click_timestamp": "datetime64[ns]",
        }
    )
)

clicks.describe(include="all", datetime_is_numeric=True)

100%|██████████| 385/385 [00:39<00:00,  9.72it/s]


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
count,2988181,2988181,2988181,2.988181e+06,2988181,2988181,2988181,2988181,2988181,2988181,2988181,2988181
unique,322897,1048594,NaN,NaN,46033,NaN,3,5,8,11,28,7
top,5890,1507563657895091,NaN,NaN,160974,NaN,4 - Web,1 - Tablet,17 - Firefox OS,1,25,2
freq,1232,124,NaN,NaN,37213,NaN,2904478,1823162,1738138,2852406,804985,1602601
mean,NaN,NaN,2017-10-08 14:17:08.013157120,3.901885e+00,NaN,2017-10-08 14:51:05.106516736,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,2017-10-01 02:37:03,2.000000e+00,NaN,2017-10-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2017-10-04 13:35:52,2.000000e+00,NaN,2017-10-04 14:20:52,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2017-10-08 20:09:00,3.000000e+00,NaN,2017-10-08 20:35:30,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2017-10-11 19:16:54,4.000000e+00,NaN,2017-10-11 19:43:24,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,2017-10-17 03:36:19,1.240000e+02,NaN,2017-11-13 20:04:14,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
if Path(DATA_PATH, "users.csv").exists():
    users = pd.read_csv(
        Path(DATA_PATH, "users.csv"),
        index_col="user_id",
        parse_dates=["MEAN_click_timestamp"],
        dtype={
            "user_id": "category",
            "COUNT_clicks": "int",
            "TOP_click_environment": "category",
            "TOP_click_deviceGroup": "category",
            "TOP_click_os": "category",
            "TOP_click_country": "category",
            "TOP_click_region": "category",
            "TOP_click_referrer_type": "category",
        },
    )
else:
    users = (
        clicks.reset_index()
        .groupby(["user_id"])
        .agg(
            COUNT_clicks=("index", "count"),
            MEAN_click_timestamp=("click_timestamp", "mean"),
            TOP_click_environment=("click_environment", lambda x: x.mode()[0]),
            TOP_click_deviceGroup=("click_deviceGroup", lambda x: x.mode()[0]),
            TOP_click_os=("click_os", lambda x: x.mode()[0]),
            TOP_click_country=("click_country", lambda x: x.mode()[0]),
            TOP_click_region=("click_region", lambda x: x.mode()[0]),
            TOP_click_referrer_type=("click_referrer_type", lambda x: x.mode()[0]),
        )
    )
    users.to_csv(Path(DATA_PATH, "users.csv"))

users = reduce_dataframe_memory_usage(
    users.astype(
        {
            "COUNT_clicks": "int",
            "MEAN_click_timestamp": "datetime64[ns]",
        }
    )
)

users.describe(include="all", datetime_is_numeric=True)

,COUNT_clicks,MEAN_click_timestamp,TOP_click_environment,TOP_click_deviceGroup,TOP_click_os,TOP_click_country,TOP_click_region,TOP_click_referrer_type
count,322897.000000,322897,322897,322897,322897,322897,322897,322897
unique,NaN,NaN,3,4,8,11,28,7
top,NaN,NaN,4 - Web,1 - Tablet,17 - Firefox OS,1,25,2
freq,NaN,NaN,313195,178033,168975,309375,88147,199292
mean,9.254285,2017-10-08 12:16:32.858684416,NaN,NaN,NaN,NaN,NaN,NaN
min,2.000000,2017-10-01 03:00:16,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.000000,2017-10-05 22:11:28,NaN,NaN,NaN,NaN,NaN,NaN
50%,4.000000,2017-10-08 11:04:36.333333248,NaN,NaN,NaN,NaN,NaN,NaN
75%,10.000000,2017-10-10 19:33:16.037037056,NaN,NaN,NaN,NaN,NaN,NaN
max,1232.000000,2017-10-31 18:05:18.750000128,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
articles_dummy = pd.get_dummies(articles.astype({"created_at_ts": "int"}))
articles_std = StandardScaler().fit_transform(articles_dummy)

pca = PCA(n_components=100, random_state=42)
pca.fit(articles_std)


px.line(
    x=range(1, len(pca.explained_variance_ratio_) + 1),
    y=pca.explained_variance_ratio_,
    title="PCA - Explained variance ratio",
    labels={"x": "Number of components", "y": "Explained variance ratio"},
)

In [7]:
articles_pca = PCA(n_components=30, random_state=42).fit_transform(articles_std)

articles_features_df = pd.DataFrame(
    articles_pca,
    columns=[f"pca_{i}" for i in range(articles_pca.shape[1])],
    index=articles.index,
)
articles_features_df

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29
article_id,,,,,,,,,,,,,,,,,,,,,
0,-1.904217,6.079083,1.537285,3.803356,-1.437661,0.300164,0.414924,1.153095,-0.518384,-1.580218,...,1.186689,-4.108220,3.301937,0.918489,-1.535456,-1.747083,0.543124,2.522120,2.387463,0.993695
1,-5.347632,-4.245111,-2.292849,3.663295,0.028957,3.627188,1.995660,-2.004237,0.694689,-3.107613,...,-3.968054,-3.666591,1.585111,-0.779830,0.929112,2.157147,0.023333,-0.421091,-0.850012,0.786145
2,-3.696948,-0.448311,-6.074376,-0.811967,-0.613763,4.426367,-1.923632,-0.186425,-3.575581,-3.095207,...,-1.839188,-4.814834,0.285541,1.491495,-0.198318,4.186979,0.819105,1.177056,0.781487,1.032065
3,-1.240908,-7.545892,-5.644173,1.761396,-2.215087,-1.675357,0.931872,-2.734797,0.209416,-4.369502,...,-3.149297,-2.173498,-2.259873,0.901336,2.609307,4.849199,-0.057921,2.123891,1.282785,2.014930
4,-1.214284,-2.792408,-2.859503,0.522262,-3.329005,5.188360,0.040037,0.870475,-0.855172,-2.437184,...,-3.854811,-2.851128,0.209985,3.347756,2.067148,6.896545,0.728921,0.651588,2.506581,1.285317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364042,2.081019,-7.314419,8.566584,-0.309293,1.824057,-1.800729,1.581976,0.381579,1.649158,-1.735606,...,0.250462,-0.567057,1.093721,-1.732896,0.253907,0.917926,-0.174882,-1.241205,-0.525054,-2.392277
364043,0.884327,-2.186223,6.148030,3.325564,14.074189,4.150520,1.042025,3.243719,2.587823,-1.147876,...,2.079845,4.947366,1.913572,-1.558480,-1.636125,-0.145713,-2.455539,-0.170820,0.826395,0.722621
364044,1.386953,-9.478225,3.365114,3.498520,2.230976,0.297417,-2.376159,-1.420322,3.050007,4.068927,...,-2.362487,1.176679,-1.667474,-2.308578,-0.216476,3.113620,-0.296673,-1.725380,-0.065677,-0.915070


In [8]:
num_clusters = np.logspace(1, 7, num=30, base=2).astype(int)
inertia = []
calinski_harabasz = []
davies_bouldin = []
for k in tqdm(num_clusters):
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=42).fit(articles_pca)
    inertia.append(kmeans.inertia_)
    calinski_harabasz.append(calinski_harabasz_score(articles_pca, kmeans.labels_))
    davies_bouldin.append(davies_bouldin_score(articles_pca, kmeans.labels_))

100%|██████████| 30/30 [00:38<00:00,  1.28s/it]


In [9]:
fig = px.line(
    x=num_clusters,
    y=inertia,
    title="Inertia",
    labels={"x": "Number of clusters", "y": "Inertia"},
)
fig.show()

In [10]:
fig = px.line(
    x=num_clusters,
    y=calinski_harabasz,
    title="Calinski-Harabasz",
    labels={"x": "Number of clusters", "y": "Calinski-Harabasz"},
)
fig.show()

In [11]:
fig = px.line(
    x=num_clusters,
    y=davies_bouldin,
    title="Davies-Bouldin",
    labels={"x": "Number of clusters", "y": "Davies-Bouldin"},
)
fig.show()

In [12]:
articles_clusters = MiniBatchKMeans(n_clusters=40, random_state=42).fit_predict(
    articles_pca
)

articles_clusters

array([29, 39, 24, ...,  5, 37,  1], dtype=int32)

In [13]:
users_dummy = pd.get_dummies(users.astype({"MEAN_click_timestamp": "int"}))
users_std = StandardScaler().fit_transform(users_dummy)

pca = PCA(n_components=50, random_state=42)
pca.fit(users_std)


px.line(
    x=range(1, len(pca.explained_variance_ratio_) + 1),
    y=pca.explained_variance_ratio_,
    title="PCA - Explained variance ratio",
    labels={"x": "Number of components", "y": "Explained variance ratio"},
)

In [14]:
users_pca = PCA(n_components=10, random_state=42).fit_transform(users_std)

users_features_df = pd.DataFrame(
    users_pca,
    columns=[f"pca_{i}" for i in range(users_pca.shape[1])],
    index=users.index,
)
users_features_df

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9
user_id,,,,,,,,,,
0,1.699164,-0.397796,-0.446491,-1.127296,0.663800,0.483216,0.715027,0.924418,-0.468858,-1.221070
1,-1.655270,0.228214,0.374687,-1.113772,0.300500,-0.940414,-0.742251,0.540289,0.539775,-0.487475
10,-2.056614,0.091305,-0.038654,2.007981,-0.688700,-0.454136,0.822261,-2.200997,-1.495055,0.471279
100,1.601323,-1.244009,-1.178582,1.780181,-0.127551,1.288546,-0.374174,-0.314391,0.227090,0.544389
1000,1.874431,-1.109712,-0.910541,-1.110672,-0.504261,1.237397,-0.263376,-0.316927,0.462357,0.520649
...,...,...,...,...,...,...,...,...,...,...
99995,-1.697570,0.243863,0.353953,-1.424103,-0.581714,-0.291931,0.416355,0.257984,0.681366,0.670509
99996,-1.649689,0.228985,0.373032,-1.153030,0.248697,-0.953786,-0.674827,0.588817,0.570464,-0.528344
99997,-1.729577,0.249486,0.358148,-1.269522,-0.346360,-0.214800,0.165836,0.072995,0.574554,0.876379


In [15]:
num_clusters = np.logspace(1, 7, num=30, base=2).astype(int)
inertia = []
calinski_harabasz = []
davies_bouldin = []
for k in tqdm(num_clusters):
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=42).fit(users_pca)
    inertia.append(kmeans.inertia_)
    calinski_harabasz.append(calinski_harabasz_score(users_pca, kmeans.labels_))
    davies_bouldin.append(davies_bouldin_score(users_pca, kmeans.labels_))

100%|██████████| 30/30 [00:17<00:00,  1.69it/s]


In [16]:
fig = px.line(
    x=num_clusters,
    y=inertia,
    title="Inertia",
    labels={"x": "Number of clusters", "y": "Inertia"},
)
fig.show()

In [17]:
fig = px.line(
    x=num_clusters,
    y=calinski_harabasz,
    title="Calinski-Harabasz",
    labels={"x": "Number of clusters", "y": "Calinski-Harabasz"},
)
fig.show()

In [18]:
fig = px.line(
    x=num_clusters,
    y=davies_bouldin,
    title="Davies-Bouldin",
    labels={"x": "Number of clusters", "y": "Davies-Bouldin"},
)
fig.show()

In [19]:
users_clusters = MiniBatchKMeans(n_clusters=40, random_state=42).fit_predict(users_pca)

users_clusters

array([28, 11, 25, ..., 33, 17,  3], dtype=int32)

In [20]:
def get_ratings_from_clicks(clicks):
    count_user_article_clicks = (
        clicks.reset_index()
        .groupby(["user_id", "click_article_id"])
        .agg(
            COUNT_user_article_clicks=("index", "count"),
        )
    )

    count_user_clicks = (
        clicks.reset_index()
        .groupby(["user_id"])
        .agg(
            COUNT_user_clicks=("index", "count"),
        )
    )

    ratings = count_user_article_clicks.join(count_user_clicks, on="user_id")
    ratings["rating"] = (
        ratings["COUNT_user_article_clicks"] / ratings["COUNT_user_clicks"]
    )

    ratings = reduce_dataframe_memory_usage(
        ratings["rating"]
        .reset_index()
        .rename({"click_article_id": "article_id"}, axis=1)
    )

    return ratings


ratings = get_ratings_from_clicks(clicks)

ratings_sample = ratings.sample(frac=0.01, random_state=42)

ratings.describe(include="all", datetime_is_numeric=True)

,user_id,article_id,rating
count,2950710,2950710,2.950710e+06
unique,322897,46033,NaN
top,5890,160974,NaN
freq,1048,34145,NaN
mean,NaN,NaN,1.094304e-01
std,NaN,NaN,1.295019e-01
min,NaN,NaN,8.116883e-04
25%,NaN,NaN,2.702703e-02
50%,NaN,NaN,5.882353e-02
75%,NaN,NaN,1.250000e-01


In [21]:
from lightfm.data import Dataset


dataset = Dataset()

dataset.fit(
    users=list(users.index),
    items=list(articles.index),
    user_features=users_features_df.columns.tolist(),
    item_features=articles_features_df.columns.tolist(),
)

dataset.interactions_shape()

(interactions, weights) = dataset.build_interactions(
    (row.user_id, row.article_id, row.rating) for row in tqdm(ratings.itertuples())
)

user_features = dataset.build_user_features(
    (user_id, dict(user_feats))
    for user_id, user_feats in tqdm(users_features_df.iterrows())
)

item_features = dataset.build_item_features(
    (article_id, dict(article_feats))
    for article_id, article_feats in tqdm(articles_features_df.iterrows())
)

2950710it [00:14, 198747.62it/s]
322897it [00:32, 9944.13it/s] 
364047it [01:18, 4647.08it/s]


In [22]:
model = LightFM(loss="warp", random_state=42)
model.fit(
    interactions=interactions,
    user_features=user_features,
    item_features=item_features,
    sample_weight=weights,
    epochs=20,
    num_threads=8,
    verbose=True,
)

Epoch: 100%|██████████| 20/20 [21:59<00:00, 65.96s/it]


In [23]:
user_id = "5890"

user_idx = dataset._user_id_mapping[user_id]

pred = model.predict(
    user_ids=user_idx,
    item_ids=list(dataset._item_id_mapping.values()),
)

In [24]:
pred_df = pd.DataFrame(
    {
        "article_id": list(dataset._item_id_mapping.keys()),
        "pred": pred,
    }
)

pred_df.sort_values("pred", ascending=False).reset_index(drop=True).head(20)

,article_id,pred
0,272143,42.181442
1,123909,38.007896
2,168623,37.624229
3,336221,36.337982
4,336245,34.041584
5,183176,33.625839
6,96210,33.018124
7,160417,32.678463
8,331116,31.216785
9,336223,30.963295


In [25]:
def aggregate_articles(articles):
    return articles.groupby(lambda x: True).agg(
        {
            col: "mean"
            if is_numeric_dtype(articles.dtypes[col])
            else lambda x: x.mode()[0]
            for col in articles.columns
        }
    )


def get_user_interest(user_id, clicks, articles, strategy="last_click"):
    user_id = str(user_id)

    if strategy == "last_click":
        last_clicked_article_id = (
            clicks.query("user_id == @user_id")
            .sort_values("click_timestamp", ascending=False)
            .reset_index(drop=True)
            .iloc[0]["click_article_id"]
        )
        interest = articles.query("article_id == @last_clicked_article_id")

    elif strategy == "last_session":
        last_session_id = (
            clicks.query("user_id == @user_id")
            .sort_values("click_timestamp", ascending=False)
            .reset_index(drop=True)
            .iloc[0]["session_id"]
        )
        session_article_ids = clicks.query("session_id == @last_session_id")[
            "click_article_id"
        ]
        interest = aggregate_articles(
            articles.query("article_id in @session_article_ids")
        ).drop(["article_id"], axis=1)

    elif strategy == "all_clicks":
        all_article_ids = clicks.query("user_id == @user_id")["click_article_id"]
        interest = aggregate_articles(
            articles.query("article_id in @all_article_ids")
        ).drop(["article_id"], axis=1)

    else:
        raise NotImplementedError

    return interest


def prepare_for_scale(articles, category_id):
    articles_copy = articles.drop(["article_id", "similarity"], axis=1, errors="ignore")
    articles_copy["category_id"] = articles_copy["category_id"].apply(
        lambda x: category_id if int(x) == category_id else 0
    )
    articles_copy["created_at_ts"] = articles_copy["created_at_ts"].apply(
        lambda x: x.value
    )

    return articles_copy


def get_closest_articles(interest, articles, n=10):
    category_id = interest["category_id"].iloc[0]

    scaler = StandardScaler()
    articles_std = scaler.fit_transform(prepare_for_scale(articles, category_id))
    interest_std = scaler.transform(prepare_for_scale(interest, category_id))

    articles = articles.copy()
    articles["similarity"] = cosine_similarity(interest_std, articles_std)[0]

    return (
        articles.sort_values("similarity", ascending=False).iloc[:n],
        scaler,
        articles_std,
        interest_std,
    )


def get_collaborative_reco(user_id, model, dataset, n=10):
    user_idx = dataset._user_id_mapping[user_id]

    pred = model.predict(
        user_ids=user_idx,
        item_ids=list(dataset._item_id_mapping.values()),
    )

    pred_df = pd.DataFrame(
        {
            "article_id": list(dataset._item_id_mapping.keys()),
            "pred": pred,
        }
    )

    return list(pred_df.sort_values("pred", ascending=False).head(n)["article_id"])

In [26]:
user_id = "5890"

interest = get_user_interest(
    user_id, clicks, articles.reset_index(), strategy="all_clicks"
)
category_id = interest["category_id"].iloc[0]

closest_article_ids = get_collaborative_reco(user_id, model, dataset)

scaler = StandardScaler()
articles_std = scaler.fit_transform(prepare_for_scale(articles, category_id))
articles_sample_std = scaler.transform(prepare_for_scale(articles_sample, category_id))

interest_std = scaler.transform(prepare_for_scale(interest, category_id))

closest_articles = articles.iloc[closest_article_ids].reset_index()
closest_articles_std = scaler.transform(
    prepare_for_scale(closest_articles, category_id)
)

In [27]:
pca = PCA(n_components=2)
articles_pca = pca.fit_transform(articles_sample_std)
interest_pca = pca.transform(interest_std)
closest_articles_pca = pca.transform(closest_articles_std)


# Plot the data in the PCA space
fig = px.scatter(
    x=articles_pca[:, 0],
    y=articles_pca[:, 1],
    color=articles_sample["category_id"],
    symbol=articles_sample["category_id"],
    title="PCA 2D",
    opacity=0.3,
    width=1200,
    height=800,
)
fig.add_scatter(
    x=interest_pca[:, 0],
    y=interest_pca[:, 1],
    mode="markers",
    marker=dict(color="green", size=30),
    text=f"User interest \n user_id: {user_id} \n category_id: {interest['category_id'].iloc[0]}",
)
fig.add_scatter(
    x=closest_articles_pca[:, 0],
    y=closest_articles_pca[:, 1],
    mode="markers",
    marker=dict(color=list(range(len(closest_articles_pca))), size=20),
    text=[
        f"rank: {i} / article_id: {a.article_id} / category_id: {a.category_id}"
        for i, a in enumerate(closest_articles.itertuples())
    ],
)
fig.show()

In [28]:
tsne = TSNE(n_components=2)
articles_tsne = tsne.fit_transform(
    np.concatenate((articles_sample_std, closest_articles_std, interest_std))
)

interest_tsne = articles_tsne[-1:]
articles_tsne = articles_tsne[:-1]

closest_articles_tsne = articles_tsne[-len(closest_articles) :]
articles_tsne = articles_tsne[: -len(closest_articles)]


# Plot the data in the t-SNE space
fig = px.scatter(
    x=articles_tsne[:, 0],
    y=articles_tsne[:, 1],
    color=articles_sample["category_id"],
    symbol=articles_sample["category_id"],
    title="t-SNE 2D",
    opacity=0.3,
    width=1200,
    height=800,
)
fig.add_scatter(
    x=interest_tsne[:, 0],
    y=interest_tsne[:, 1],
    mode="markers",
    marker=dict(color="green", size=30),
    text=f"User interest \n user_id: {user_id} \n category_id: {interest['category_id'].iloc[0]}",
)
fig.add_scatter(
    x=closest_articles_tsne[:, 0],
    y=closest_articles_tsne[:, 1],
    mode="markers",
    marker=dict(color=list(range(len(closest_articles_tsne))), size=20),
    text=[
        f"rank: {i} / article_id: {a.article_id} / category_id: {a.category_id}"
        for i, a in enumerate(closest_articles.itertuples())
    ],
)
fig.show()

/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [29]:
users_last_click = (
    clicks.reset_index()
    .rename(columns={"index": "click_id"})
    .sort_values(by="click_timestamp")
    .groupby(["user_id"])
    .last()
)

X = get_ratings_from_clicks(clicks.drop(list(users_last_click["click_id"])))
y_true = dict(users_last_click["click_article_id"])

In [30]:
(interactions, weights) = dataset.build_interactions(
    (row.user_id, row.article_id, row.rating) for row in tqdm(X.itertuples())
)

model = LightFM(loss="warp", random_state=42)
model.fit(
    interactions=interactions,
    user_features=user_features,
    item_features=item_features,
    sample_weight=weights,
    epochs=20,
    num_threads=8,
    verbose=True,
)

2629418it [00:12, 212996.25it/s]
Epoch: 100%|██████████| 20/20 [20:55<00:00, 62.79s/it]


In [31]:
test_sample = random.sample(list(y_true.keys()), k=1000)

y_pred = dict(
    {
        user_id: get_collaborative_reco(user_id, model, dataset, n=1000)
        for user_id in tqdm(test_sample)
    }
)

100%|██████████| 1000/1000 [04:09<00:00,  4.01it/s]


In [32]:
def score_reco(y_true, y_pred):
    score = 0
    count = 0
    for user_id, pred_article_ids in y_pred.items():
        if user_id not in y_true.keys():
            logging.info(f"User {user_id} not found in true values")
            continue

        true_article_id = str(y_true[user_id])

        if true_article_id not in pred_article_ids:
            logging.info(
                f"Article {true_article_id} not found in predictions for user {user_id}"
            )
            continue

        rank = pred_article_ids.index(true_article_id) + 1
        score += 1 / rank
        count += 1

    # In range [0 , 1], higher is better
    return score / count


def mean_rank(y_true, y_pred):
    sum = 0
    count = 0
    for user_id, pred_article_ids in y_pred.items():
        if user_id not in y_true.keys():
            logging.info(f"User {user_id} not found in true values")
            continue

        true_article_id = str(y_true[user_id])

        if true_article_id not in pred_article_ids:
            logging.info(
                f"Article {true_article_id} not found in predictions for user {user_id}"
            )
            continue

        rank = pred_article_ids.index(true_article_id) + 1
        sum += rank
        count += 1

    # In range [1 , +Inf[, lower is better
    return sum / count


def mean_average_precision(y_true, y_pred, articles, k=10):
    average_precision = 0
    for user_id, pred_article_ids in y_pred.items():
        if user_id not in y_true.keys():
            logging.warning(f"User {user_id} not found in true values")
            continue

        true_category_id = articles.iloc[int(y_true[user_id])].category_id
        pred_categories = articles.iloc[
            [int(id) for id in pred_article_ids[:k]]
        ].category_id

        average_precision = (
            len(pred_categories[pred_categories == true_category_id]) / k
        )

    return average_precision / len(y_pred)

In [34]:
print(f"Score : {score_reco(y_true, y_pred)}")
print(f"Mean Rank : {mean_rank(y_true, y_pred)}")
print(
    f"Mean Average Precision : {mean_average_precision(y_true, y_pred, articles, k=1000)}"
)

Score : 0.03606015565909779
Mean Rank : 224.3379746835443
Mean Average Precision : 8e-06
